# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откроем и изучим файлы
Импортируем необходимые библиотеки и функции. Откроем датасет.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


Данные не трубуют предобработки. Для машинного обучение необходимо только разбить датасет на признаки и целевые значения.

## Разбейте данные на выборки
Разобьём данные на признаки и целевые значения. Также выделим обучающую, валидационную и тестовую выборки в пропорциях 0.6, 0.2, 0.2 соответсвенно.

In [4]:
features = df.drop(['is_ultra'],axis=1)
target = df['is_ultra']

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.40, random_state=12345)

features_test, features_valid, target_test, target_valid = train_test_split(
    features_valid, target_valid, test_size=0.50, random_state=12345)

Проверим, как коррелируют признаки. 

In [5]:
df.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


Сильная корреляция наблюдается только между минутами и звонками, как и должно быть. Все признаки одинаково коррелируют с целевым.

## Исследуйте модели

Посмотри, как решающие деревья, случайный лес и логистическая регрессия справляются с данной задачей классификации.
### Решающие деревья
Меняя глубину дерева, подберем модель с наилучшими результатами на валидационной выборке. 

In [6]:
best_tree_model = None
best_tree_result = 0
best_tree_depth = 0
for depth in range(1,10):
    model = DecisionTreeClassifier(random_state=12345,max_depth=depth)
    model.fit(features_train,target_train) 
    predictions = model.predict(features_valid)
    result = accuracy_score(predictions,target_valid)
    if result > best_tree_result:
        best_tree_model = model
        best_tree_result = result
        best_tree_depth = depth

In [7]:
print('Процент правильных предсказанных ответов: {:.2%} \nГлубина дерева: {}'.format(best_tree_result, best_tree_depth))

Процент правильных предсказанных ответов: 79.94% 
Глубина дерева: 7


### Случайный лес
Меняя количество деревьев и их глубину, подберем модель с наилучшими результатами на валидационной выборке. 

In [8]:
best_forest_model = None
best_forest_result = 0
best_forest_depth = 0
best_forest_est = 0
for est in range(10,15):
    for depth in range(3,11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train,target_train)
        result = model.score(features_valid, target_valid)
        if result > best_forest_result:
            best_forest_model = model
            best_forest_result = result
            best_forest_depth = depth
            best_forest_est = est

In [9]:
print('Процент правильных предсказанных ответов: {:.2%}\nГлубина деревьев: {} \nКоличество деревьев: {}'.format(
    best_forest_result, best_forest_depth,best_forest_est))

Процент правильных предсказанных ответов: 81.49%
Глубина деревьев: 9 
Количество деревьев: 14


### Логистическая регрессия


In [10]:
logistic_model = LogisticRegression(random_state=12345)
logistic_model.fit(features_train,target_train)
logistic_result = logistic_model.score(features_valid, target_valid)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [11]:
print('Процент правильных предсказанных ответов: {:.2%}'.format(logistic_result))

Процент правильных предсказанных ответов: 74.03%


## Проверим модель на тестовой выборке
Проверим accuracy на тестовой выборке для Логистической регрессии, случайного леса и решающих деревьев соответсвенноо:

In [12]:
log_score = logistic_model.score(features_test, target_test)

In [13]:
forest_score = best_forest_model.score(features_test, target_test)

In [14]:
predictions = best_tree_model.predict(features_test)
tree_score = accuracy_score(predictions,target_test)

In [15]:
print('Accuracy на тестовой выборке для различных моделей:\n',
      pd.Series(data={'Логистическая регрессия':log_score,
                      'Случайный лес':forest_score,
                      'Решающие деревья':tree_score}))

Accuracy на тестовой выборке для различных моделей:
 Логистическая регрессия    0.758942
Случайный лес              0.790047
Решающие деревья           0.782271
dtype: float64


Необходимо добиться acceracy не менее 75%.
Лучший результат у случайного леса - 79%, хотя решающие деревья тоже показали хороший результат - 78%. 
У логистической регрессии результат близок к пороговому 76%.


<div class="alert alert-info">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Круто было бы еще иметь сводную табличку с результатами. Было бы наглядно:)
</div>

## Проверим модели на адекватность
Для поверки адекватности используем функцию log_loss, учитывающую вероятность предсказания класса в каждом случае, а не только правильно предсказанные классы.

In [16]:
print('log_loss на тестовой выборке для различных моделей:\n \n',
      pd.Series(data={'Логистическая регрессия':log_loss(target_test,best_tree_model.predict_proba(features_test)),
                      'Случайный лес':log_loss(target_test,best_forest_model.predict_proba(features_test)),
                      'Решающие деревья':log_loss(target_test,logistic_model.predict_proba(features_test))}))

log_loss на тестовой выборке для различных моделей:
 
 Логистическая регрессия    1.565888
Случайный лес              0.636502
Решающие деревья           0.569699
dtype: float64


Модель можно считать вменяемой, если log_loss находится в пределах [0.5,1] и чем ближе значение к 1, тем лучше модель. Этому условию соответствуют случайный лес и логистическая регрессия.

## Вывод
Обучив модели решающих деревьев, случайного леса и логистической регрессии, мы определили их оптимальные гиперпараметры для данной задачи.  
Точность моделей на тестовой выборке:  

- Случайный лес - 79%,
- Решающие деревья - 78%. 
- Логистическая регрессия - 76%.

Проверка моделей на адекватность показала, что результаты предсказаний моделей Случайного и Логистической регрессии наиболее адекватны. То есть вероятность предсказать правильный ответ в каждом отдельном случае для правильных ответов выше.  
При финальном выборе модели стоит учитывать скорость их работы:
- Быстрые модели: Решающие деревья и Логистическая регрессия.
- Медленные модели: Случайный лес.